In [2]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn import decomposition
from sklearn.ensemble import *
from sklearn.feature_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *

import matplotlib.pyplot as plt

import csv
import time
import copy
import os
import tempfile
import xgboost

In [44]:
data = pd.read_csv('train_final.csv')
data.head()

,avg_travel_time,last_att,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,ls_0_in_out_diff,...,ls_5_117,ls_6_103,ls_6_118,ls_6_121,ls_8_101,ls_8_113,ls_7_106,ls_7_116,ls_7_122,ls_9_121
0,85.405,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
1,58.050,85.405,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
2,67.895,58.050,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
3,67.895,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
4,56.870,67.895,0.248927,0.25,0.608333,0.44,0.529644,0.767123,0.0,-1,...,1,0,0,0,0,0,0,0,0,0


In [45]:
train_data, val_data = train_test_split(data, test_size=0.20, shuffle=False)
train_target = np.array(train_data.pop('avg_travel_time'))
val_target = np.array(val_data.pop('avg_travel_time'))

In [52]:
model = xgboost.XGBRegressor(gamma=0,               
                 learning_rate=0.05,
                 max_depth=200,
                 min_child_weight=1,
                 n_estimators=60,                                                                    
                 reg_alpha=0.01,
                 reg_lambda=0.001) 
model.fit(train_data,train_target,
        eval_set=[(train_data, train_target), (val_data, val_target)],
        eval_metric='mae',
        verbose=True)

[03:58:36] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:109.341	validation_1-mae:174.969
[1]	validation_0-mae:103.877	validation_1-mae:167.082
[2]	validation_0-mae:98.6869	validation_1-mae:159.591
[3]	validation_0-mae:93.756	validation_1-mae:152.501
[4]	validation_0-mae:89.0714	validation_1-mae:145.833
[5]	validation_0-mae:84.6204	validation_1-mae:139.6
[6]	validation_0-mae:80.3924	validation_1-mae:133.729
[7]	validation_0-mae:76.3756	validation_1-mae:128.182
[8]	validation_0-mae:72.5601	validation_1-mae:122.959
[9]	validation_0-mae:68.9345	validation_1-mae:118.116
[10]	validation_0-mae:65.4903	validation_1-mae:113.604
[11]	validation_0-mae:62.2185	validation_1-mae:109.393
[12]	validation_0-mae:59.1092	validation_1-mae:105.452
[13]	validation_0-mae:56.1563	validation_1-mae:101.782
[14]	validation_0-mae:53.3506	validation_1-mae:98.3484
[15]	validation_0-mae:50.6854	validation_1-mae:95.1602
[16]	validation_0

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.05, max_delta_step=0,
             max_depth=200, min_child_weight=1, missing=None, n_estimators=60,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0.01, reg_lambda=0.001, scale_pos_weight=1, seed=None,
             silent=None, subsample=1, verbosity=1)

In [53]:
validation = model.predict(val_data)
mape_score_v = np.mean(np.abs((val_target - validation) / val_target))
mape_score_v

0.2969518462781045

In [54]:
train = model.predict(train_data)
mape_score_t = np.mean(np.abs((train_target - train) / train_target))
mape_score_t

0.04594414721394298

In [49]:
test_data = pd.read_csv('test_final.csv')
test_data.head()

,avg_travel_time,last_att,pressure,sea_pressure,wind_direction,wind_speed,temperature,rel_humidity,precipitation,ls_0_in_out_diff,...,ls_5_117,ls_6_103,ls_6_118,ls_6_121,ls_8_101,ls_8_113,ls_7_106,ls_7_116,ls_7_122,ls_9_121
0,41.097143,41.097143,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
1,43.681667,41.097143,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
2,68.024286,43.681667,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
3,52.608333,68.024286,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0
4,56.165556,52.608333,0.746781,0.745763,0.355556,0.213333,0.367589,0.657534,0.0,-1,...,1,0,0,0,0,0,0,0,0,0


In [50]:
test_label = test_data.pop('avg_travel_time')

In [55]:
result = model.predict(test_data)
mape_score = np.mean(np.abs((test_label - result) / test_label))
mape_score

0.30467465956765677